YT Vid: https://www.youtube.com/watch?v=cO5g5qLrLSo

# Import Libs

In [2]:
import random
import gym

In [3]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [4]:
states

4

In [5]:
actions

2

### Random actions

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} Score: {score}")

Episode 1 Score: 23.0
Episode 2 Score: 27.0
Episode 3 Score: 23.0
Episode 4 Score: 10.0
Episode 5 Score: 15.0
Episode 6 Score: 29.0
Episode 7 Score: 39.0
Episode 8 Score: 10.0
Episode 9 Score: 15.0
Episode 10 Score: 29.0


## Deep Learning Model

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [14]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [15]:
model = build_model(states, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## Reinforcementlearning agent

In [17]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [21]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=1000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [22]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
dqn.fit(env, nb_steps=1000, visualize=False, verbose = 1)

Training for 1000 steps ...
Interval 1 (0 steps performed)
  999/10000 [=>............................] - ETA: 2:05 - reward: 1.0000done, took 14.056 seconds


In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 100 episodes ...
Episode 1: reward: 164.000, steps: 164
Episode 2: reward: 175.000, steps: 175
Episode 3: reward: 142.000, steps: 142
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 150.000, steps: 150
Episode 6: reward: 161.000, steps: 161
Episode 7: reward: 155.000, steps: 155
Episode 8: reward: 148.000, steps: 148
Episode 9: reward: 152.000, steps: 152
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 144.000, steps: 144
Episode 12: reward: 134.000, steps: 134
Episode 13: reward: 148.000, steps: 148
Episode 14: reward: 172.000, steps: 172
Episode 15: reward: 150.000, steps: 150
Episode 16: reward: 152.000, steps: 152
Episode 17: reward: 150.000, steps: 150
Episode 18: reward: 154.000, steps: 154
Episode 19: reward: 156.000, steps: 156
Episode 20: reward: 194.000, steps: 194
Episode 21: reward: 145.000, steps: 145
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 137.000, steps: 137
Episode 24: reward: 166.000, steps: 166
Episode 25: reward: 

In [28]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: 200.000, steps: 200
